## Graph_RAG
***
使用知识图谱来增强检索

数据准备

In [2]:
! pip install -qU graph_rag_example_helpers


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


下载测试文档

In [3]:
from graph_rag_example_helpers.datasets.animals import fetch_documents
animals = fetch_documents()

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [5]:
! pip install "langchain-graph-retriever[chroma]"


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


简单处理下数据

In [6]:
for doc in animals:
    keys_to_delete = []
    for key, value in doc.metadata.items():
        if isinstance(value, dict):
            keys_to_delete.append(key)
    for key in keys_to_delete:
        del doc.metadata[key]

print(animals)

[Document(id='aardvark', metadata={'type': 'mammal', 'number_of_legs': 4, 'keywords': ['burrowing', 'nocturnal', 'ants', 'savanna'], 'habitat': 'savanna', 'tags': [{'a': 5, 'b': 7}, {'a': 8, 'b': 10}]}, page_content='the aardvark is a nocturnal mammal known for its burrowing habits and long snout used to sniff out ants.'), Document(id='albatross', metadata={'type': 'bird', 'number_of_legs': 2, 'keywords': ['seabird', 'wingspan', 'ocean'], 'habitat': 'marine', 'tags': [{'a': 5, 'b': 8}, {'a': 8, 'b': 10}]}, page_content='the albatross is a large seabird with the longest wingspan of any bird, allowing it to glide effortlessly over oceans.'), Document(id='alligator', metadata={'type': 'reptile', 'number_of_legs': 4, 'keywords': ['reptile', 'jaws', 'wetlands'], 'diet': 'carnivorous'}, page_content='alligators are large reptiles with powerful jaws and are commonly found in freshwater wetlands.'), Document(id='alpaca', metadata={'type': 'mammal', 'number_of_legs': 4, 'keywords': ['wool', 'do

创建一个向量数据库测试

In [7]:
from langchain_chroma.vectorstores import Chroma
from langchain_graph_retriever.transformers import ShreddingTransformer


vector_store = Chroma.from_documents(
    documents=list(ShreddingTransformer().transform_documents(animals)),
    embedding=embeddings,
    collection_name="animals",
)

创建一个基于知识图谱的检索，在这个检索里检索器从与查询最匹配的单个动物开始，然后遍历到具有相同habitat或origin的动物

In [8]:
from graph_retriever.strategies import Eager
from langchain_graph_retriever import GraphRetriever

traversal_retriever = GraphRetriever(
    store = vector_store,
    edges = [("habitat", "habitat"), ("origin", "origin")],
    strategy = Eager(k=5, start_k=1, max_depth=2),
)

提问

In [9]:
results = traversal_retriever.invoke("what animals could be found near a capybara?")

for doc in results:
    print(f"{doc.id}: {doc.page_content}")

capybara: capybaras are the largest rodents in the world and are highly social animals.
heron: herons are wading birds known for their long legs and necks, often seen near water.
crocodile: crocodiles are large reptiles with powerful jaws and a long lifespan, often living over 70 years.
frog: frogs are amphibians known for their jumping ability and croaking sounds.
duck: ducks are waterfowl birds known for their webbed feet and quacking sounds.


整合到问答链中

In [10]:
from langchain_deepseek import ChatDeepSeek
import os
llm = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    api_base=os.environ.get("DEEPSEEK_API_BASE"),
)

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
"""Answer the question based only on the context provided.

Context: {context}

Question: {question}
the final answer should be use chinese
"""

)

def format_docs(docs):
    return "\n\n".join(f"text: {doc.page_content} metadata: {doc.metadata}" for doc in docs)

chain = (
    {"context": traversal_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
chain.invoke("what animals could be found near a capybara?")

'根据提供的上下文，以下动物可能会出现在水豚附近，因为它们都生活在湿地环境中：\n\n1. 鹭 (Herons)  \n2. 鳄鱼 (Crocodiles)  \n3. 青蛙 (Frogs)  \n4. 鸭子 (Ducks)  \n\n这些动物与水豚共享湿地栖息地，因此可能会在附近出现。'